The cell below is just for creating paths

In [2]:
import os
import shutil

# General Purpose Paths
data_small_path = os.path.join('data','data_small.csv')
data_big_path = os.path.join('data','data_big.csv')
results_path = 'results'

# Ex1 - Paths
output_sql_ex1_path = os.path.join(results_path,'Ex1_SQL.csv')
# Ex2 - Paths
output_sql_ex2_path = os.path.join(results_path,'Ex2_SQL.csv')
# Ex3 - Paths
output_sql_ex3_path = os.path.join(results_path,'Ex3_SQL.csv')

# Create result directory from scratch
if not os.path.exists(results_path):
    os.makedirs(results_path)

# Exercise 1

Create an inverted index with the following structure:

- Continent - Count of occurences

In [5]:
%%time
from pyspark.sql import Row, SparkSession
from IPython.display import display

spark = SparkSession.builder.master('local[*]').appName('Ex1').getOrCreate()
sc = spark.sparkContext

try :
    lines = sc.textFile(data_big_path)
    continent_occurences = lines.map( lambda line : line.split(',') ) \
                   .map( lambda arr : Row( continent = arr[5], occurences = int(arr[9])))
    continent_occurences_df = spark.createDataFrame( continent_occurences )
    continent_occurences_df.createOrReplaceTempView("continents")
    
    query = ("SELECT continent,SUM(occurences) as n_occurences from continents "
             "Group by continent")
    InvIdxContinentOccurencesDF = spark.sql(query)
    InvIdxContinentOccurencesDF.toPandas().to_csv(output_sql_ex1_path,index=False)
    sc.stop()
except:
    print(err)
    sc.stop()




CPU times: user 60 ms, sys: 40 ms, total: 100 ms
Wall time: 21.8 s


# Exercise 2

Create an inverted index with the following structure:
- disaster_type : regions

In [7]:
%%time
from pyspark.sql import Row, SparkSession
from IPython.display import display

spark = SparkSession.builder.master('local[*]').appName('Ex2').getOrCreate()
sc = spark.sparkContext

try :
    lines = sc.textFile(data_big_path)
    disasters = lines.map( lambda line : line.split(',') ) \
                   .map( lambda arr : Row( disaster = arr[3], region = arr[6]))
    disasterDF = spark.createDataFrame(disasters)
    disasterDF.createOrReplaceTempView("disasters")
    
    query = ("SELECT disaster, collect_set(region) as regions from disasters "
             "Group by disaster")
    InvIdxDisasterRegionDF = spark.sql(query)
    InvIdxDisasterRegionDF.toPandas().to_csv(output_sql_ex2_path,index=False)
    sc.stop()
except:
    print(err)
    sc.stop()

CPU times: user 50 ms, sys: 30 ms, total: 80 ms
Wall time: 23.4 s


# Exercise 3

Using an inverted index, solve: <br>

What are the probabilities of getting injured or dying in a natural disaster of type T in the continent C
during decade D

In [11]:
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
import pandas as pd


def fillaffected(inj,death,affected):
    if float(affected)<float(inj)+float(death):
        affected=float(inj)+float(death)
    return affected

def fillnan(x):
    if x=="":
        return 0
    else:
        return x

spark = SparkSession.builder.master('local[*]').appName('Ex3').getOrCreate()
sc = spark.sparkContext
spark.udf.register("fillaffected",fillaffected)
spark.udf.register("fillnan",fillnan)
try :
    lines = sc.textFile(data_big_path)
    
    data = lines.filter( lambda line : len(line) > 0 )   \
                        .map( lambda line : line.split(',') ) \
                        .map( lambda arr : Row( dec = arr[0][:-1]+'0', DT = arr[3], continent = arr[5], \
                                               inj = arr[11], deaths = arr[10], affected = arr[14]))
    DF = spark.createDataFrame( data )
    
    DF.createOrReplaceTempView("ini")
    
    DF=spark.sql("select DT,continent,fillnan(dec) as dec,fillnan(inj) as inj,fillnan(deaths) as deaths,fillnan(affected) as affected from ini")
    DF.createOrReplaceTempView("dis")
    
    sumaff=spark.sql("select DT,continent,dec,inj,deaths,fillaffected(inj,deaths,affected) as affected from dis")
    sumaff.createOrReplaceTempView("dis")
    sumall=spark.sql("select continent,DT,dec,ROUND(sum(inj)/(sum(affected)+0.001),3) as inj_prob,ROUND(sum(deaths)/(sum(affected)+0.001),3) as death_prob from dis group by DT,continent,dec")

    sumall.toPandas().to_csv(output_sql_ex3_path,index=False)
    sc.stop()
except Exception as e:
    print(e)
    sc.stop()